<a href="https://colab.research.google.com/github/HaziqSaeed/ExpertSystem/blob/master/Learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset

# Define a transformation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load the entire CIFAR-10 training dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

# Class indices for 'cat' and 'dog'
class_indices = [3, 5]  # Cat: 3, Dog: 5

# Filter out only the images with 'cat' and 'dog' labels
filtered_indices = [i for i, label in enumerate(trainset.targets) if label in class_indices]

# Subset the dataset to include only cat and dog images
filtered_trainset = Subset(trainset, filtered_indices)

# Create a DataLoader for the filtered dataset
trainloader = DataLoader(filtered_trainset, batch_size=4, shuffle=True, num_workers=2)

# Check the filtered dataset size
print(f"Filtered dataset size: {len(filtered_trainset)}")

# Example: Iterate through the DataLoader and print out a batch of images
data_iter = iter(trainloader)
images, labels = next(data_iter) # Use next(data_iter) instead of data_iter.next()

# Display the labels to confirm filtering (should only be 3 or 5)
print("Batch labels:", labels)

Files already downloaded and verified
Filtered dataset size: 10000
Batch labels: tensor([3, 5, 3, 5])


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)  # 3 input channels (RGB), 16 output channels, 3x3 convolution
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)  # 2x2 max pooling
        self.fc1 = nn.Linear(32 * 8 * 8, 128)  # Fully connected layer
        self.fc2 = nn.Linear(128, 2)  # Output layer with 2 classes (cat and dog)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 8 * 8)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the network
net = SimpleCNN()


In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()  # Loss function
optimizer = optim.Adam(net.parameters(), lr=0.001)  # Optimizer


In [ ]:
# Training loop
for epoch in range(5):  # Number of epochs

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # Convert labels to binary (0 for cat, 1 for dog)
        labels = torch.tensor([0 if label == 3 else 1 for label in labels])

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = net(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')


[Epoch 1, Batch 100] loss: 0.690
[Epoch 1, Batch 200] loss: 0.671
[Epoch 1, Batch 300] loss: 0.680
[Epoch 1, Batch 400] loss: 0.657
[Epoch 1, Batch 500] loss: 0.671
[Epoch 1, Batch 600] loss: 0.652
[Epoch 1, Batch 700] loss: 0.651
[Epoch 1, Batch 800] loss: 0.653
[Epoch 1, Batch 900] loss: 0.649
[Epoch 1, Batch 1000] loss: 0.637
[Epoch 1, Batch 1100] loss: 0.632
[Epoch 1, Batch 1200] loss: 0.613
[Epoch 1, Batch 1300] loss: 0.636
[Epoch 1, Batch 1400] loss: 0.618
[Epoch 1, Batch 1500] loss: 0.609
[Epoch 1, Batch 1600] loss: 0.635
[Epoch 1, Batch 1700] loss: 0.590
[Epoch 1, Batch 1800] loss: 0.590
[Epoch 1, Batch 1900] loss: 0.585
[Epoch 1, Batch 2000] loss: 0.622
[Epoch 1, Batch 2100] loss: 0.601
[Epoch 1, Batch 2200] loss: 0.592
[Epoch 1, Batch 2300] loss: 0.596
[Epoch 1, Batch 2400] loss: 0.596
[Epoch 1, Batch 2500] loss: 0.592
[Epoch 2, Batch 100] loss: 0.589
[Epoch 2, Batch 200] loss: 0.579
[Epoch 2, Batch 300] loss: 0.568
[Epoch 2, Batch 400] loss: 0.534
[Epoch 2, Batch 500] loss: 

In [ ]:
# Load test dataset (filtered in a similar way to training dataset)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
filtered_test_indices = [i for i, label in enumerate(testset.targets) if label in class_indices]
filtered_testset = Subset(testset, filtered_test_indices)
testloader = DataLoader(filtered_testset, batch_size=4, shuffle=False, num_workers=2)

# Test the network
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        labels = torch.tensor([0 if label == 3 else 1 for label in labels])
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the cat and dog images: {100 * correct / total:.2f}%')


Files already downloaded and verified
Accuracy of the network on the cat and dog images: 74.15%
